In [ ]:
import pandas as pd
import numpy as np

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Untitled folder/cik_list.xlsx')
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [ ]:
len(data)

152

In [ ]:
data['SECFNAME'] ='https://www.sec.gov/Archives/'+data['SECFNAME']

In [ ]:
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...


In [ ]:
import requests 

def get_data(link):
    hdr = {'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Mobile Safari/537.36'}

    req = requests.get(link,headers=hdr)
    content = req.content.decode('utf-8')
  
    return content

reports = []

for url in data['SECFNAME']:
    reports.append(get_data(url))


In [ ]:
import pickle

In [ ]:
pickle.dump(reports,open('report.pkl','wb'))


In [ ]:
print(len(reports[0]))
print(reports[0])

In [ ]:
import re
from nltk.corpus import stopwords
# update = re.sub('[^a-zA-Z]'," ",reports[0].strip())
# valid = [words for words in update.split() if len(words)>2]
# valid = [words for words in valid if words.lower() not in stopwords.words("english")]


In [ ]:
word_reports = []

for i in range(len(reports)):
    update = re.sub('[^a-zA-Z]'," ",reports[i].strip())
    valid = [words for words in update.split() if len(words)>2]
    valid = [words for words in valid if words.lower() not in stopwords.words("english")]
    word_reports.append(valid)
    if i%5==0:
      print(i)


len(word_reports)

65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150


152

In [ ]:
word_reports

In [ ]:
import pickle
pickle.dump(word_reports,open('/content/drive/MyDrive/Untitled folder/word_reports.pkl','wb'))

In [ ]:
dictionary = pd.read_csv('/content/drive/MyDrive/Untitled folder/LoughranMcDonald_MasterDictionary_2020.csv')

In [ ]:
dictionary.head()

,Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Complexity,Syllables,Source
0,AARDVARK,1,312,1.422050e-08,1.335201e-08,3.700747e-06,96,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.367356e-10,8.882163e-12,9.362849e-09,1,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,9,4.102067e-10,1.200533e-10,5.359747e-08,7,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,15,6.836779e-10,4.080549e-10,1.406914e-07,14,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,8009,3.650384e-07,3.798698e-07,3.523914e-05,1058,0,0,0,0,0,0,0,0,3,12of12inf


In [ ]:
positive_words = dictionary[dictionary.Positive!=0]["Word"].to_numpy()

In [ ]:
negative_words = dictionary[dictionary.Negative!=0]["Word"].to_numpy()

In [ ]:
len(positive_words),len(negative_words)

(354, 2355)

In [ ]:
def positive_score(words,positive_words):
  result = 0
  for word in words:
    if word.upper() in positive_words:
      result+=1
  return result

def negative_score(words,negative_words):
  result = 0
  for word in words:
    if word.upper() in negative_words:
      result+=1
  return result

In [ ]:
pos_scores = []
for i in range(len(word_reports)):
  pos_scores.append(positive_score(word_reports[i],positive_words))

In [ ]:
neg_scores = []
for i in range(len(word_reports)):
  neg_scores.append(negative_score(word_reports[i],negative_words))
  if i%10==0:
    print(i)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150


In [ ]:
data["positive_score"] = pos_scores
data["negative_score"] = neg_scores

In [ ]:
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,1065,2907
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,585,1428
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,2,8
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...,368,1404
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,3,8


In [ ]:
new_data.to_excel('/content/drive/MyDrive/Untitled folder/new.xls')

In [ ]:
new_data = pd.read_excel('/content/drive/MyDrive/Untitled folder/new.xls')
new_data.head()

,Unnamed: 0,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score
0,0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,1065,2907
1,1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,585,1428
2,2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,2,8
3,3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...,368,1404
4,4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,3,8


In [ ]:
new_data["SECFNAME"]= data['SECFNAME']

In [ ]:
new_data.head()

,Unnamed: 0,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score
0,0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,1065,2907
1,1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,585,1428
2,2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,2,8
3,3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,368,1404
4,4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,3,8


In [ ]:
new_data['polarity_score'] = (new_data['positive_score']-new_data['negative_score'])/((new_data['positive_score']+new_data['negative_score'])+0.000001)

In [ ]:
new_data.head()

,Unnamed: 0,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,polarity_score
0,0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,1065,2907,-0.463746
1,1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,585,1428,-0.418778
2,2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,2,8,-0.600000
3,3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,368,1404,-0.584650
4,4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,3,8,-0.454545


In [ ]:
new_data.to_excel('/content/drive/MyDrive/Untitled folder/new.xls')

In [2]:
import pickle
reports = pickle.load(open('/content/drive/MyDrive/Untitled folder/report.pkl','rb'))
word_reports = pickle.load(open('/content/drive/MyDrive/Untitled folder/word_reports.pkl','rb'))

len(reports),len(word_reports)

(152, 152)

In [ ]:
import nltk

sent = nltk.sent_tokenize(reports[0])
len(sent)

3963

In [ ]:
def avg_sent(report,word_report):
  sent = nltk.sent_tokenize(report)
  return (len(word_report)/len(sent))


In [ ]:
avg_length = []
for i in range(len(reports)):
  avg_length.append(avg_sent(reports[i],word_reports[i]))

In [ ]:

data = pd.read_excel('/content/drive/MyDrive/Untitled folder/new.xls')
data['average sentence length'] = avg_length
data.head()

,Unnamed: 0,Unnamed: 0.1,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,polarity_score,average sentence length
0,0,0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,1065,2907,-0.463746,22.362604
1,1,1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,585,1428,-0.418778,22.948317
2,2,2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,2,8,-0.600000,26.950000
3,3,3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,368,1404,-0.584650,18.139694
4,4,4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,3,8,-0.454545,25.920000


In [9]:
def is_syllable(word):
  if word.endswith('es') or word.endswith('ed'):
    return False
  count = 0 
  for letter in word:
    if letter.lower() in 'aeiou':
      count+=1
      if count>2:
        return True
  return False

def complex_word(word_report):
  count =0 
  for word in word_report:
    if is_syllable(word):
      count+=1
  return count

In [13]:
percent_complex = []
for word_report in word_reports:
  percent_complex.append(complex_word(word_report))

In [18]:
data["percentage_of_complex_words"] = percent_complex

In [19]:
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,polarity_score,average sentence length,percentage_of_complex_words
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,1065,2907,-0.463746,22.362604,0.446837
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,585,1428,-0.418778,22.948317,0.464725
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,2,8,-0.600000,26.950000,0.391466
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,368,1404,-0.584650,18.139694,0.467787
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,3,8,-0.454545,25.920000,0.365741


In [20]:
data['fog_index'] = 0.4*(data['average sentence length']+data['percentage_of_complex_words'])
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,polarity_score,average sentence length,percentage_of_complex_words,fog_index
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,1065,2907,-0.463746,22.362604,0.446837,9.123776
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,585,1428,-0.418778,22.948317,0.464725,9.365217
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,2,8,-0.600000,26.950000,0.391466,10.936586
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,368,1404,-0.584650,18.139694,0.467787,7.442992
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,3,8,-0.454545,25.920000,0.365741,10.514296


In [23]:
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,polarity_score,average_sentence_length,percentage_of_complex_words,fog_index
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,1065,2907,-0.463746,22.362604,0.446837,9.123776
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,585,1428,-0.418778,22.948317,0.464725,9.365217
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,2,8,-0.600000,26.950000,0.391466,10.936586
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,368,1404,-0.584650,18.139694,0.467787,7.442992
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,3,8,-0.454545,25.920000,0.365741,10.514296


In [24]:
def complex_word(word_report):
  count =0 
  for word in word_report:
    if is_syllable(word):
      count+=1
  return count

comp_word = []
for word_report in word_reports:
  comp_word.append(complex_word(word_report))

In [25]:
data['complex_word_count'] =comp_word
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,polarity_score,average_sentence_length,percentage_of_complex_words,fog_index,complex_word_count
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,1065,2907,-0.463746,22.362604,0.446837,9.123776,39600
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,585,1428,-0.418778,22.948317,0.464725,9.365217,26619
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,2,8,-0.600000,26.950000,0.391466,10.936586,211
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,368,1404,-0.584650,18.139694,0.467787,7.442992,21078
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,3,8,-0.454545,25.920000,0.365741,10.514296,237


In [26]:
word_count = []
for words in word_reports:
  word_count.append(len(words))

data["word_count"] = word_count
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,polarity_score,average_sentence_length,percentage_of_complex_words,fog_index,complex_word_count,word_count
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,1065,2907,-0.463746,22.362604,0.446837,9.123776,39600,88623
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,585,1428,-0.418778,22.948317,0.464725,9.365217,26619,57279
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,2,8,-0.600000,26.950000,0.391466,10.936586,211,539
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,368,1404,-0.584650,18.139694,0.467787,7.442992,21078,45059
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,3,8,-0.454545,25.920000,0.365741,10.514296,237,648


In [27]:
data.to_excel("/content/drive/MyDrive/Untitled folder/add.xls")

In [43]:
constraining_words = pd.read_excel('/content/drive/MyDrive/Untitled folder/constraining_dictionary.xlsx')
constraining_words = constraining_words['Word'].to_numpy()

uncertainity_words = pd.read_excel('/content/drive/MyDrive/Untitled folder/uncertainty_dictionary.xlsx')
uncertainity_words = uncertainity_words['Word'].to_numpy()

In [46]:
def constrain_words(word_report):
  count = 0 
  for word in word_report:
    if word in constraining_words:
      count+=1
  return count

def uncertain_words(word_report):
  count = 0 
  for word in word_report:
    if word in uncertainity_words:
      count+=1
  return count

In [49]:
constrain, uncertain = [],[]

for words in word_reports:
  constrain.append(constrain_words(words))
  uncertain.append(uncertain_words(words))

In [50]:
data['uncertainity_score'] = uncertain
data["constraining_score"] = constrain
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,polarity_score,average_sentence_length,percentage_of_complex_words,fog_index,complex_word_count,word_count,uncertainity_score,constraining_score
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,1065,2907,-0.463746,22.362604,0.446837,9.123776,39600,88623,13,79
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,585,1428,-0.418778,22.948317,0.464725,9.365217,26619,57279,16,86
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,2,8,-0.600000,26.950000,0.391466,10.936586,211,539,0,0
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,368,1404,-0.584650,18.139694,0.467787,7.442992,21078,45059,13,44
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,3,8,-0.454545,25.920000,0.365741,10.514296,237,648,0,0


In [55]:
data.to_excel('/content/drive/MyDrive/Untitled folder/new.xls')

In [52]:
data['positive_word_proportion'] = data['positive_score']/data["word_count"]
data['negative_word_proportion'] = data['negative_score']/data["word_count"]
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,polarity_score,average_sentence_length,percentage_of_complex_words,fog_index,complex_word_count,word_count,uncertainity_score,constraining_score,positive_word_proportion,negative_word_proportion
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,1065,2907,-0.463746,22.362604,0.446837,9.123776,39600,88623,13,79,0.012017,0.032802
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,585,1428,-0.418778,22.948317,0.464725,9.365217,26619,57279,16,86,0.010213,0.024931
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,2,8,-0.600000,26.950000,0.391466,10.936586,211,539,0,0,0.003711,0.014842
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,368,1404,-0.584650,18.139694,0.467787,7.442992,21078,45059,13,44,0.008167,0.031159
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,3,8,-0.454545,25.920000,0.365741,10.514296,237,648,0,0,0.004630,0.012346


In [54]:
data['uncertainity_word_proportion'] = data['uncertainity_score']/data["word_count"]
data['constraining_word_proportion'] = data['constraining_score']/data["word_count"]
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,polarity_score,average_sentence_length,percentage_of_complex_words,fog_index,complex_word_count,word_count,uncertainity_score,constraining_score,positive_word_proportion,negative_word_proportion,uncertainity_word_proportion,constraining_word_proportion
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,1065,2907,-0.463746,22.362604,0.446837,9.123776,39600,88623,13,79,0.012017,0.032802,0.000147,0.000891
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,585,1428,-0.418778,22.948317,0.464725,9.365217,26619,57279,16,86,0.010213,0.024931,0.000279,0.001501
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,2,8,-0.600000,26.950000,0.391466,10.936586,211,539,0,0,0.003711,0.014842,0.000000,0.000000
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,368,1404,-0.584650,18.139694,0.467787,7.442992,21078,45059,13,44,0.008167,0.031159,0.000289,0.000976
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,3,8,-0.454545,25.920000,0.365741,10.514296,237,648,0,0,0.004630,0.012346,0.000000,0.000000


In [71]:
data['constrainig_words_whole_report'] = data["constraining_score"]

In [72]:
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,polarity_score,average_sentence_length,percentage_of_complex_words,fog_index,complex_word_count,word_count,uncertainity_score,constraining_score,positive_word_proportion,negative_word_proportion,uncertainity_word_proportion,constraining_word_proportion,constrainig_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,1065,2907,-0.463746,22.362604,0.446837,9.123776,39600,88623,13,79,0.012017,0.032802,0.000147,0.000891,79
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,585,1428,-0.418778,22.948317,0.464725,9.365217,26619,57279,16,86,0.010213,0.024931,0.000279,0.001501,86
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,2,8,-0.600000,26.950000,0.391466,10.936586,211,539,0,0,0.003711,0.014842,0.000000,0.000000,0
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,368,1404,-0.584650,18.139694,0.467787,7.442992,21078,45059,13,44,0.008167,0.031159,0.000289,0.000976,44
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,3,8,-0.454545,25.920000,0.365741,10.514296,237,648,0,0,0.004630,0.012346,0.000000,0.000000,0


In [74]:
data.set_index("CIK",inplace=True)

In [75]:
data.head()

,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,polarity_score,average_sentence_length,percentage_of_complex_words,fog_index,complex_word_count,word_count,uncertainity_score,constraining_score,positive_word_proportion,negative_word_proportion,uncertainity_word_proportion,constraining_word_proportion,constrainig_words_whole_report
CIK,,,,,,,,,,,,,,,,,,,,
3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,1065,2907,-0.463746,22.362604,0.446837,9.123776,39600,88623,13,79,0.012017,0.032802,0.000147,0.000891,79
3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,585,1428,-0.418778,22.948317,0.464725,9.365217,26619,57279,16,86,0.010213,0.024931,0.000279,0.001501,86
3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,2,8,-0.600000,26.950000,0.391466,10.936586,211,539,0,0,0.003711,0.014842,0.000000,0.000000,0
3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,368,1404,-0.584650,18.139694,0.467787,7.442992,21078,45059,13,44,0.008167,0.031159,0.000289,0.000976,44
3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,3,8,-0.454545,25.920000,0.365741,10.514296,237,648,0,0,0.004630,0.012346,0.000000,0.000000,0


In [76]:
data.to_excel('report.xls')